In [23]:
import pandas as pd

df = pd.read_csv("muse_v3.csv")

In [24]:
df.head()

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal


removing whitespace from columsn if any

In [25]:
df.columns = df.columns.str.strip()

In [26]:
emotion_cols = ['valence_tags', 'arousal_tags', 'dominance_tags']
df[emotion_cols] = df[emotion_cols].apply(pd.to_numeric, errors='coerce')

In [27]:
df.dropna(subset=['spotify_id'] + emotion_cols, inplace=True)

In [28]:
df_clean = df[df['track'].apply(lambda x: isinstance(x, str) and x.isascii())]
df_clean = df_clean[df_clean['artist'].apply(lambda x: isinstance(x, str) and x.isascii())]

In [29]:
len(df_clean)

58886

In [30]:
df_clean.to_csv("muse_v3_clean_english.csv", index=False)

In [ ]:
import pandas as pd
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=config.CLIENT_ID,
    client_secret=config.CLIENT_SECRET
))

df = pd.read_csv("muse_v3_clean_english.csv")

df_subset = df[df['spotify_id'].notna()].head(100).copy()

batch_size = 50
track_ids = df_subset['spotify_id'].tolist()

updated_tracks = []
updated_artists = []

for i in range(0, len(track_ids), batch_size):
    batch_ids = track_ids[i:i + batch_size]
    try:
        
        tracks_info = sp.tracks(batch_ids)['tracks']
        for track in tracks_info:
            if track: 
                updated_tracks.append(track['name'])
                updated_artists.append(track['artists'][0]['name'])
            else:
                updated_tracks.append(None)
                updated_artists.append(None)
    except Exception as e:
        print(f"Error in batch starting at index {i}: {e}")
        
        updated_tracks.extend([None]*len(batch_ids))
        updated_artists.extend([None]*len(batch_ids))
    time.sleep(0.2)  # Avoid rate limit

original_tracks = df.loc[df['spotify_id'].notna(), 'track'].head(100).tolist()
original_artists = df.loc[df['spotify_id'].notna(), 'artist'].head(100).tolist()

final_tracks = [new if new is not None else orig for new, orig in zip(updated_tracks, original_tracks)]
final_artists = [new if new is not None else orig for new, orig in zip(updated_artists, original_artists)]

df_subset.loc[:, 'track'] = final_tracks
df_subset.loc[:, 'artist'] = final_artists

df_subset.to_csv("muse_v3_spotify_names_100.csv", index=False)
print("✅ Updated 100 records saved to 'muse_v3_spotify_names_100.csv'")


✅ Updated 100 records saved to 'muse_v3_spotify_names_100.csv'
